In [0]:
dbutils.widgets.text("folder_name" , "")

In [0]:
p_folder_name = dbutils.widgets.get("folder_name")

### Data Reading

In [0]:
spark.read.format("parquet").load("abfss://source@stretaildataproj.dfs.core.windows.net/orders/orders-second.parquet").printSchema()

### AutoLoader Implementation

In [0]:
df = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format" ,"parquet")\
    .option("cloudFiles.schemalocation" , f"abfss://bronze@stretaildataproj.dfs.core.windows.net/checkpoints/checkpoint_{p_folder_name}")\
    .load(f"abfss://source@stretaildataproj.dfs.core.windows.net/{p_folder_name}/")



In [0]:
df.writeStream.format("parquet")\
    .outputMode("append")\
    .option("checkpointLocation", f"abfss://bronze@stretaildataproj.dfs.core.windows.net/checkpoints/checkpoint_{p_folder_name}")\
    .option("path", f"abfss://bronze@stretaildataproj.dfs.core.windows.net/{p_folder_name}")\
    .trigger(once=True)\
    .start()

In [0]:
spark.read.format("parquet").load(f"abfss://bronze@stretaildataproj.dfs.core.windows.net/{p_folder_name}").display()